In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
from tkinter import *
from PIL import Image, ImageTk
import pickle
import numpy as np
import pandas as pd
import shap

/home/bammpy/housepred/vm/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Initialize the Tkinter application
root = Tk()
root.title("House Price Prediction System")
root.geometry("700x700")

''

In [3]:
# Define global variables for model, feature names, and background data
model = None
feature_names = None
background_data = None

# Load the model, feature names, and background data
try:
    with open('house_price_model.pkl', 'rb') as file:
        data = pickle.load(file)
        if isinstance(data, dict):
            model = data['model']
            feature_names = data['feature_names']
            background_data = data.get('background_data', None)  # Use .get() for backward compatibility
        else:
            model = data
            feature_names = ['bedrooms', 'location', 'stories', 'furnishings', 'bathrooms', 
                             'guestroom', 'electricity', 'water', 'internet', 'waste_disposal', 
                             'proximity_amenities', 'urbanization']
            background_data = None
    print("Model and feature names loaded: house_price_model.pkl")
except FileNotFoundError:
    print("⚠️ No existing model found. Please train one using train_model.py.")

Model and feature names loaded: house_price_model.pkl


In [4]:
# Entry field for Number of Stories
Label(root, text="Number of Stories:").grid(row=2, column=0, sticky=W, padx=10, pady=5)
entry_stories = Entry(root)
entry_stories.grid(row=2, column=1, padx=10, pady=5)

# Bedrooms entry
Label(root, text="Bedrooms:").grid(row=0, column=0, padx=10, pady=5)
entry_bedrooms = Entry(root)
entry_bedrooms.grid(row=0, column=1, padx=10, pady=5)

# Location Entry
Label(root, text="Location:").grid(row=1, column=0, sticky=W, padx=10, pady=5)
location_var = StringVar(root)
location_var.set("Offroad")
OptionMenu(root, location_var, "Near the Road", "Offroad").grid(row=1, column=1, padx=10, pady=5)

# Furnishing Status Entry
Label(root, text="Furnishings Status:").grid(row=3, column=0, sticky=W, padx=10, pady=5)
furnishings_var = StringVar(root)
furnishings_var.set("Fully-furnished")
OptionMenu(root, furnishings_var, "Fully-furnished", "Furnished", "Semi-furnished").grid(row=3, column=1, padx=10, pady=5)

# Bathrooms Entry
Label(root, text="Number of Bathrooms:").grid(row=4, column=0, sticky=W, padx=10, pady=5)
bathrooms_entry = Entry(root)
bathrooms_entry.grid(row=4, column=1, padx=10, pady=5)

# Availability of Utilities (checkboxes)
Label(root, text="Availability of Utilities:").grid(row=6, column=0, sticky=W, padx=10, pady=5)
electricity_var = IntVar()
water_var = IntVar()
internet_var = IntVar()
waste_disposal_var = IntVar()
Checkbutton(root, text="Electricity", variable=electricity_var).grid(row=6, column=1, sticky=W)
Checkbutton(root, text="Water", variable=water_var).grid(row=7, column=1, sticky=W)
Checkbutton(root, text="Internet", variable=internet_var).grid(row=8, column=1, sticky=W)
Checkbutton(root, text="Waste Disposal", variable=waste_disposal_var).grid(row=9, column=1, sticky=W)

# Guestroom Entry
Label(root, text="Guestroom (Yes/No):").grid(row=5, column=0, sticky=W, padx=10, pady=5)
guestroom_var = StringVar(root)
guestroom_var.set("No")
OptionMenu(root, guestroom_var, "Yes", "No").grid(row=5, column=1, padx=10, pady=5)

# Proximity to Amenities Entry
Label(root, text="Proximity to Amenities:").grid(row=10, column=0, sticky=W, padx=10, pady=5)
proximity_amenities_var = StringVar(root)
proximity_amenities_var.set("Yes")
Radiobutton(root, text="Yes", variable=proximity_amenities_var, value="Yes").grid(row=10, column=1, sticky=W)
Radiobutton(root, text="No", variable=proximity_amenities_var, value="No").grid(row=11, column=1, sticky=W)

# Urbanization Level Entry
Label(root, text="Urbanization Level:").grid(row=12, column=0, sticky=W, padx=10, pady=5)
urbanization_level_var = StringVar(root)
urbanization_level_var.set("High")
Radiobutton(root, text="High", variable=urbanization_level_var, value="High").grid(row=12, column=1, sticky=W)
Radiobutton(root, text="Low", variable=urbanization_level_var, value="Low").grid(row=13, column=1, sticky=W)

# Label to display the predicted price (Moved to the right side)
predicted_price_label = Label(root, text="Predicted Price will be shown here.", font=("Arial", 12, "bold"))
predicted_price_label.grid(row=0, column=3, padx=20, pady=10, sticky=W)

# User Query Section
Label(root, text="Ask a Question:").grid(row=4, column=3, sticky=W, padx=20, pady=5)
user_query_entry = Entry(root, width=40)
user_query_entry.grid(row=5, column=3, padx=20, pady=5, sticky=W)

# Label to display query response (Moved to the right)
query_response_label = Label(root, text="", wraplength=300, justify="left", font=("Arial", 10))
query_response_label.grid(row=6, column=3, padx=20, pady=10, sticky=W)

# Label to display feature explanation (Moved to the right)
explanation_label = Label(root, text="", wraplength=300, justify="left", font=("Arial", 10))
explanation_label.grid(row=1, column=3, padx=20, pady=10, sticky=W)

# Add image at bottom-right corner under "Ask AI"
try:
    img = Image.open("hg.jpg")  # Replace with your image file
    img = img.resize((100, 100), Image.Resampling.LANCZOS)  # Resize to fit
    photo = ImageTk.PhotoImage(img)
    image_label = Label(root, image=photo)
    image_label.grid(row=8, column=3, padx=20, pady=10, sticky=SE)  # Below "Ask" and "Ask Return"
except FileNotFoundError:
    print("⚠️ Image 'hg.jpg' not found. Proceeding without it.")

# Prediction Function (unchanged)
def predict_price():
    global model, feature_names, background_data
    if model is None or feature_names is None:
        predicted_price_label.config(text="Error: Model not loaded. Run train_model.py first.")
        print("Error: Model or feature names not loaded.")
        return
    try:
        bedrooms = int(entry_bedrooms.get())
        stories = int(entry_stories.get())
        bathrooms = int(bathrooms_entry.get())
        location = 1 if location_var.get() == "Near the Road" else 0
        furnishings = 2 if furnishings_var.get() == "Fully-furnished" else 1 if furnishings_var.get() == "Furnished" else 0
        guestroom = 1 if guestroom_var.get() == "Yes" else 0
        proximity_amenities = 1 if proximity_amenities_var.get() == "Yes" else 0
        urbanization = 1 if urbanization_level_var.get() == "High" else 0
        electricity = electricity_var.get()
        water = water_var.get()
        internet = internet_var.get()
        waste_disposal = waste_disposal_var.get()

        input_data = pd.DataFrame([[
            bedrooms, location, stories, furnishings, bathrooms, guestroom,
            electricity, water, internet, waste_disposal, proximity_amenities, urbanization
        ]], columns=feature_names)

        predicted_price = model.predict(input_data)[0]

        if background_data is not None:
            explainer = shap.Explainer(model, background_data)
        else:
            explainer = shap.Explainer(model, input_data)
        shap_values = explainer(input_data)
        impact_scores = shap_values.values[0]
        sorted_features = sorted(zip(feature_names, impact_scores), key=lambda x: abs(x[1]), reverse=True)
        top_features = sorted_features[:3]
        explanation_text = "Top 3 Factors Influencing Price:\n"
        for feature, score in top_features:
            explanation_text += f" - {feature}: {'Increases' if score > 0 else 'Decreases'} price by {abs(score):,.2f}\n"

        predicted_price_label.config(text=f"Predicted Price: Ksh {predicted_price:,.2f}")
        explanation_label.config(text=explanation_text)
        print(f"Input: {input_data.iloc[0].to_list()}")
        print(f"Predicted Price: Ksh {predicted_price:,.2f}")
        for feature, score in zip(feature_names, impact_scores):
            print(f"{feature}: {score:.2f}")

        plt.figure(figsize=(10, 5))
        sns.barplot(x=[x[0] for x in sorted_features], y=[x[1] for x in sorted_features], hue=[x[0] for x in sorted_features], palette="coolwarm", legend=False)
        plt.xlabel("Features")
        plt.ylabel("SHAP Value Impact")
        plt.title("Feature Importance in House Price Prediction")
        plt.xticks(rotation=45)
        plt.show()

    except Exception as e:
        predicted_price_label.config(text="Error: Invalid input or prediction failed.")
        print(f"Error: {e}")

# Query Handling (unchanged)
def handle_query():
    query = user_query_entry.get().lower()
    responses = {
        "furnishing": "Furnishing status affects the price. Fully-furnished houses tend to be more expensive.",
        "location": "Houses near the road tend to have a higher price due to accessibility.",
        "utilities": "Having all utilities like water, electricity, and internet increases the house value.",
        "bedrooms": "The more bedrooms a house has, the higher the price tends to be.",
        "bathrooms": "More bathrooms contribute to a higher house value, especially for large families.",
        "stories": "Multi-story houses are generally more expensive due to more space and construction costs.",
        "guestroom": "A guestroom can increase the price as it adds more space for visitors.",
        "proximity": "Being close to amenities like schools, hospitals, and shopping centers increases house value.",
        "urbanization": "High urbanization levels lead to higher property prices due to increased demand.",
        "electricity": "Stable electricity supply is a key factor in determining house prices.",
        "internet": "Houses with good internet connectivity tend to be more valuable, especially in urban areas.",
        "waste disposal": "Proper waste disposal facilities can impact the attractiveness and value of a property."
    }
    response = "I'm not sure, please ask something related to the house price features."
    for key, value in responses.items():
        if key in query:
            response = value
            break
    query_response_label.config(text=response)

# Function to clear the query input and response (unchanged)
def reset_query():
    user_query_entry.delete(0, END)
    query_response_label.config(text="")

# Buttons
Button(root, text="Predict Price", command=predict_price, bg="green", fg="white", font=("Arial", 12, "bold")).grid(row=14, column=0, columnspan=2, pady=10)
Button(root, text="Ask", command=handle_query, bg="blue", fg="white", font=("Arial", 12, "bold")).grid(row=7, column=3, padx=20, pady=10, sticky=W)

# Copyright Label (Bold & Centered)
Label(root, text="© 2025 Brightone Jeconiah Aoko. All Rights Reserved.", 
      font=("Arial", 10, "bold"), fg="gray").grid(row=30, column=0, columnspan=4, pady=10, sticky="nsew")

# Ask Return Button
Button(root, text="Ask Return", command=reset_query, bg="black", fg="white", font=("Arial", 12, "bold")).grid(row=7, column=4, padx=5, pady=10, sticky=W)

⚠️ Image 'hg.jpg' not found. Proceeding without it.


In [5]:
# Run the main loop
root.mainloop()